In [25]:
import os 
import numpy as np 
import pandas as pd 
import pymannkendall as mk 

In [26]:
class_num = 2 
file_names = os.listdir("../../event_csv/split_data/class2/")
file_name = file_names[0]
file_path = f'../../event_csv/compress_event_manhattan/class{class_num}/smooth_by_pca/compress_by_mean/{file_name}'

In [27]:
pca_data = pd.read_csv(file_path)['value']

In [51]:
data  = pca_data.copy() 
type(data), data.shape

(pandas.core.series.Series, (554,))

In [52]:
win_size = 10 
step = 5 
first = np.inf 
second = -np.inf 
flag = True  
final_win_change = None 

In [53]:
win_sum = int(
    1 + (data.shape[0]-win_size)/step 
    if data.shape[0] > win_size 
    else 0
    )

In [54]:
win_start_index = np.arange(0,win_sum*step,step=step)
# Mutation pane index [Later processing as an intermediate index in the middle pane]
win_change = np.array([])
# Current window trend [Increasing, Decreasing, No Trend]
win_trend = np.array([])

win_data = data[win_start_index[0]:win_start_index[0]+win_size]

In [55]:
# Traverson all pane
for i in range(win_sum):
    # Pane data
    win_data = data[win_start_index[i]:win_start_index[i]+win_size]
    # Add the current window trend
    win_trend = np.append(win_trend,mk.original_test(win_data)[0]) 

In [56]:
win_data.shape, win_trend.shape, win_sum

((10,), (109,), 109)

In [57]:
win_data, win_trend[-10:]

(540    8.734008
 541    6.057962
 542    5.530768
 543    5.138926
 544    4.078489
 545    2.781117
 546    3.518813
 547    1.357226
 548    0.285553
 549    0.421942
 Name: value, dtype: float64,
 array(['no trend', 'decreasing', 'decreasing', 'decreasing', 'no trend',
        'increasing', 'increasing', 'increasing', 'no trend', 'decreasing'],
       dtype='<U32'))

In [58]:
# Traversing the trend of each pane
for i in range(1,win_trend.shape[0]):
    # The previous one is not increasing, and the current is increasing, indicating that this is a mutation point
    if win_trend[i] == 'increasing' and win_trend[i-1]!='increasing':
        win_change = np.append(win_change,win_start_index[i-1])
    elif win_trend[i] == 'decreasing' and win_trend[i-1]!='decreasing':
        win_change = np.append(win_change,win_start_index[i-1])


In [59]:
win_change

array([ 20.,  50.,  70.,  90., 110., 130., 150., 170., 190., 215., 235.,
       255., 280., 305., 330., 350., 370., 390., 410., 435., 455., 475.,
       495., 515., 535.])

In [60]:
win_change = win_change + win_size/2

In [61]:
win_change

array([ 25.,  55.,  75.,  95., 115., 135., 155., 175., 195., 220., 240.,
       260., 285., 310., 335., 355., 375., 395., 415., 440., 460., 480.,
       500., 520., 540.])

In [62]:
win_change = win_change.astype(np.int32)
win_change = np.insert(win_change,0,0)
win_change = np.append(win_change,data.shape[0]-1)

In [63]:
win_change

array([  0,  25,  55,  75,  95, 115, 135, 155, 175, 195, 220, 240, 260,
       285, 310, 335, 355, 375, 395, 415, 440, 460, 480, 500, 520, 540,
       553])

In [65]:
variance = np.var(np.diff(win_change))

In [66]:
variance

np.float64(9.88905325443787)